In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

In [2]:
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  
device = torch.device(dev)

In [3]:
df = pd.read_excel("data.xlsx", sheet_name="data")
df.head()

,From Date,PM2.5,PM10,NO,NO2,NOx,NH3,SO2,CO,Ozone,Benzene
0,2019-01-01 00:00:00,0.548004,0.837675,0.386063,0.183578,0.377457,0.311067,0.099945,0.59,0.008504,0.391534
1,2019-01-01 00:15:00,0.548004,0.837675,0.447938,0.177646,0.426394,0.304088,0.088901,0.60,0.011006,0.373016
2,2019-01-01 00:30:00,0.548004,0.837675,0.385062,0.201686,0.382471,0.333500,0.088349,0.52,0.011006,0.365079
3,2019-01-01 00:45:00,0.448759,0.658317,0.440328,0.196691,0.426394,0.331007,0.085036,0.54,0.011006,0.359788
4,2019-01-01 01:00:00,0.448759,0.658317,0.422307,0.175148,0.404332,0.327517,0.083932,0.50,0.011131,0.333333


In [4]:
df = df.set_index('From Date')

In [5]:
df.shape

(70176, 10)

In [6]:
test_data_size = int(df.shape[0]/4)
test_data_size

17544

In [7]:
train_data = df[:-test_data_size]
test_data = df[-test_data_size:]
train_data.shape, test_data.shape

((52632, 10), (17544, 10))

In [8]:
train_daat = torch.FloatTensor(train_data.values).view(-1).to(device)

In [9]:
train_data_tensor = train_data.view(-1, 10)

train_data_tensor[0]

tensor([0.5480, 0.8377, 0.3861, 0.1836, 0.3775, 0.3111, 0.0999, 0.5900, 0.0085,
        0.3915], device='cuda:0')

In [10]:
device

device(type='cuda', index=0)

In [11]:
def create_inout_sequences(input_data, tw):
    inout_seq = []
    L = len(input_data)
    for i in range(L-tw):
        train_seq = input_data[i:i+tw]
        train_label = input_data[i+tw]
        inout_seq.append((train_seq ,train_label))
    return inout_seq

In [12]:
train_inout_seq = create_inout_sequences(train_data_tensor, 16)
train_inout_seq[0]

(tensor([[0.5480, 0.8377, 0.3861, 0.1836, 0.3775, 0.3111, 0.0999, 0.5900, 0.0085,
          0.3915],
         [0.5480, 0.8377, 0.4479, 0.1776, 0.4264, 0.3041, 0.0889, 0.6000, 0.0110,
          0.3730],
         [0.5480, 0.8377, 0.3851, 0.2017, 0.3825, 0.3335, 0.0883, 0.5200, 0.0110,
          0.3651],
         [0.4488, 0.6583, 0.4403, 0.1967, 0.4264, 0.3310, 0.0850, 0.5400, 0.0110,
          0.3598],
         [0.4488, 0.6583, 0.4223, 0.1751, 0.4043, 0.3275, 0.0839, 0.5000, 0.0111,
          0.3333],
         [0.4488, 0.6583, 0.3931, 0.1948, 0.3869, 0.3310, 0.0812, 0.4500, 0.0113,
          0.3360],
         [0.4488, 0.6583, 0.2729, 0.2223, 0.2968, 0.3574, 0.0812, 0.3000, 0.0114,
          0.3095],
         [0.3830, 0.5912, 0.2046, 0.2114, 0.2381, 0.3225, 0.0751, 0.2700, 0.0115,
          0.2725],
         [0.3830, 0.5912, 0.1846, 0.2092, 0.2211, 0.3038, 0.0678, 0.2500, 0.0090,
          0.2116],
         [0.3830, 0.5912, 0.1646, 0.2070, 0.2042, 0.2851, 0.0605, 0.2300, 0.0095,
         

In [13]:
seq1, labe1l = train_inout_seq[0]
seq1.shape, labe1l.shape

(torch.Size([16, 10]), torch.Size([10]))

In [14]:
class LSTM(nn.Module):
    def __init__(self, input_size=10, hidden_layer_size=100, output_size=10):
        super().__init__()
        self.hidden_layer_size = hidden_layer_size

        self.lstm = nn.LSTM(input_size, hidden_layer_size)

        self.linear = nn.Linear(hidden_layer_size, output_size)

        self.hidden_cell = (torch.zeros(1,1,self.hidden_layer_size).to(device),
                            torch.zeros(1,1,self.hidden_layer_size).to(device))

    def forward(self, input_seq):
        lstm_out, self.hidden_cell = self.lstm(input_seq.view(len(input_seq) ,1, -1), self.hidden_cell)
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

In [15]:
model = LSTM().to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [16]:
print(model)

LSTM(
  (lstm): LSTM(10, 100)
  (linear): Linear(in_features=100, out_features=10, bias=True)
)


In [17]:
epochs = 10

for i in range(epochs):
    for seq, labels in train_inout_seq:
        optimizer.zero_grad()
        model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size).to(device),
                        torch.zeros(1, 1, model.hidden_layer_size).to(device))

        y_pred = model(seq)

        single_loss = loss_function(y_pred, labels)
        single_loss.backward()
        optimizer.step()

    if i%2 == 1:
        print(f'epoch: {i:3} loss: {single_loss.item():10.8f}')

print(f'epoch: {i:3} loss: {single_loss.item():10.10f}')

epoch:   1 loss: 0.00005125
epoch:   3 loss: 0.00004820
epoch:   5 loss: 0.00004989
epoch:   7 loss: 0.00004934
epoch:   9 loss: 0.00004764
epoch:   9 loss: 0.0000476390


In [19]:
torch.save(model, "model.pt")

In [20]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

test_data_tensor = torch.FloatTensor(test_data.values).view(-1,10).to(device)

test_inout_seq = create_inout_sequences(test_data_tensor, 16)
test_inout_seq[0]


(tensor([[0.0658, 0.1263, 0.0094, 0.0952, 0.0395, 0.0962, 0.0304, 0.1000, 0.1371,
          0.0741],
         [0.0658, 0.1263, 0.0096, 0.0924, 0.0387, 0.0967, 0.0331, 0.1000, 0.1381,
          0.0741],
         [0.0658, 0.1263, 0.0098, 0.0815, 0.0351, 0.0962, 0.0387, 0.1000, 0.1381,
          0.0794],
         [0.0593, 0.1513, 0.0110, 0.0871, 0.0381, 0.0982, 0.0447, 0.1000, 0.1311,
          0.0899],
         [0.0593, 0.1513, 0.0172, 0.0871, 0.0431, 0.1017, 0.0414, 0.1100, 0.1286,
          0.0926],
         [0.0593, 0.1513, 0.0128, 0.0884, 0.0399, 0.1122, 0.0409, 0.1100, 0.1576,
          0.0952],
         [0.0593, 0.1513, 0.0124, 0.0902, 0.0401, 0.1181, 0.0375, 0.1100, 0.1746,
          0.1111],
         [0.0626, 0.1503, 0.0118, 0.0974, 0.0423, 0.1181, 0.0375, 0.1100, 0.1896,
          0.1164],
         [0.0626, 0.1503, 0.0174, 0.1140, 0.0525, 0.1171, 0.0370, 0.1200, 0.1801,
          0.1164],
         [0.0626, 0.1503, 0.0286, 0.1146, 0.0618, 0.1142, 0.0359, 0.1200, 0.2021,
         

In [21]:
test_inout_seq[0][0]

tensor([[0.0658, 0.1263, 0.0094, 0.0952, 0.0395, 0.0962, 0.0304, 0.1000, 0.1371,
         0.0741],
        [0.0658, 0.1263, 0.0096, 0.0924, 0.0387, 0.0967, 0.0331, 0.1000, 0.1381,
         0.0741],
        [0.0658, 0.1263, 0.0098, 0.0815, 0.0351, 0.0962, 0.0387, 0.1000, 0.1381,
         0.0794],
        [0.0593, 0.1513, 0.0110, 0.0871, 0.0381, 0.0982, 0.0447, 0.1000, 0.1311,
         0.0899],
        [0.0593, 0.1513, 0.0172, 0.0871, 0.0431, 0.1017, 0.0414, 0.1100, 0.1286,
         0.0926],
        [0.0593, 0.1513, 0.0128, 0.0884, 0.0399, 0.1122, 0.0409, 0.1100, 0.1576,
         0.0952],
        [0.0593, 0.1513, 0.0124, 0.0902, 0.0401, 0.1181, 0.0375, 0.1100, 0.1746,
         0.1111],
        [0.0626, 0.1503, 0.0118, 0.0974, 0.0423, 0.1181, 0.0375, 0.1100, 0.1896,
         0.1164],
        [0.0626, 0.1503, 0.0174, 0.1140, 0.0525, 0.1171, 0.0370, 0.1200, 0.1801,
         0.1164],
        [0.0626, 0.1503, 0.0286, 0.1146, 0.0618, 0.1142, 0.0359, 0.1200, 0.2021,
         0.1111],
        [0

In [ ]:
y_pred = model()